In [1]:
import os
import glob
import datetime as dt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
yf.pdr_override() # <== that's all it takes :-)

from concurrent import futures
from dateutil.relativedelta import relativedelta
from pandas_datareader import data as pdr
from scipy.stats import gaussian_kde

""" set output directory """
data_dir = "./data"
os.makedirs(data_dir, exist_ok=True)


In [2]:
""" datetime util """
now = dt.datetime.now()
lastday = now + relativedelta(months=0, days=-1)
firstday_of_this_month = dt.datetime(now.year, now.month, 1)
lastday_of_this_month = dt.datetime(now.year, now.month, 1) + relativedelta(months=1, days=-1)
firstday_of_last_month = dt.datetime(now.year, now.month, 1) + relativedelta(months=-1, days=0)
lastday_of_last_month = dt.datetime(now.year, now.month, 1) + relativedelta(months=0, days=-1)

In [3]:
""" set the download window """
start_date = "2017-01-01"
end_date = lastday.strftime('%Y-%m-%d')

In [4]:
def calculate_obv(df):
    """Calculates the on-balance volume (OBV) for a given DataFrame"""
    OBV = [0]
    for i in range(1, len(df)):
        if df['Close'][i] > df['Close'][i-1]:
            OBV.append(OBV[i-1] + df['Volume'][i])
        elif df['Close'][i] < df['Close'][i-1]:
            OBV.append(OBV[i-1] - df['Volume'][i])
        else:
            OBV.append(OBV[i-1])
    df['OBV'] = OBV
    return df

def find_divergence(df):
    """Finds bullish and bearish divergences in the OBV indicator"""
    bullish_divergence = []
    bearish_divergence = []
    
    FLAG = -1
    for i in range(1, len(df)):
        if df['Close'][i-1] < df['Close'][i] and FLAG == -1:
            bearish_divergence.append((df.index[i], df['Close'][i]))
            FLAG = 0
        elif df['Close'][i-1] > df['Close'][i] and FLAG == 0:
            bullish_divergence.append((df.index[i], df['Close'][i]))
            FLAG = -1
    return bullish_divergence, bearish_divergence

In [11]:
#highest_day_list = []

count = 0
stock_data_files = glob.glob(f"{data_dir}/*.csv")

for file in stock_data_files:
    if count >= 1:
        break
    count += 1
        
    # Load from CSV
    df = pd.read_csv(file)
    df.set_index('Date', inplace=True)
    
    df = calculate_obv(df)
    bullish_divergence, bearish_divergence = find_divergence(df)
    
    df_bullish_divergence = pd.DataFrame(bullish_divergence, columns =['Date', 'OBV'])
    df_bearish_divergence = pd.DataFrame(bearish_divergence, columns =['Date', 'OBV'])    
    
    ticker = file.split("\\")[-1].split(".")[0]
    df_bullish_divergence.to_csv(f"{ticker}_bullish_divergence_{end_date}.csv")
    df_bearish_divergence.to_csv(f"{ticker}_bearish_divergence_{end_date}.csv")

In [ ]:
from plotly.offline import plot
import plotly.graph_objs as go

df = pd.read_csv('./output.csv')

o = df['open'].astype(float)
h = df['high'].astype(float)
l = df['low'].astype(float)
c = df['Close'].astype(float)

trace = go.Candlestick(
            open=o,
            high=h,
            low=l,
            Close=c)
data = [trace]
plot(data, filename='go_candle1.html')

In [4]:
google = yf.Ticker("GOOG")
df = google.history() # period='1d', interval="1m"
df['Date'] = pd.to_datetime(df.index).date # .time 
df.set_index('Date', inplace=True) 
# Save to CSV
df.to_csv("stock_data.csv")
df.head()

data_dir = "./data"
stock_data_files = glob.glob(f"{data_dir}/*.csv")

In [ ]:



X = df.index.values
y = df['Low'].values
# The split point is the 10% of the dataframe length
offset = int(0.10*len(df))
X_train = X[:-offset]
y_train = y[:-offset]
X_test  = X[-offset:]
y_test  = y[-offset:]

In [9]:
import yfinance as yf
google = yf.Ticker("GOOG")

#df = google.history(period='1d', interval="1m")
df = google
print(df.head())

import pandas as pd

df['날짜'] = pd.to_datetime(df.index).time 

df.set_index('날짜', inplace=True) 

df.head()

X = df.index.values
y = df['Low'].values
# The split point is the 10% of the dataframe length
offset = int(0.10*len(df))
X_train = X[:-offset]
y_train = y[:-offset]
X_test  = X[-offset:]
y_test  = y[-offset:]

import matplotlib.pyplot as plt
plt.plot(range(0,len(y_train)),y_train, label='Train')
plt.plot(range(len(y_train),len(y)),y_test,label='Test')
plt.legend()
plt.show()

from plotly.offline import plot
import plotly.graph_objs as go

df = pd.read_csv('./output.csv')

o = df['open'].astype(float)
h = df['high'].astype(float)
l = df['low'].astype(float)
c = df['Close'].astype(float)

trace = go.Candlestick(
            open=o,
            high=h,
            low=l,
            Close=c)
data = [trace]
plot(data, filename='go_candle1.html')